# U-Net

### U-Net Overview
U-Net은 주로 의료 영상에서 사용되는 이미지 분할을 위한 Convolution Neural Network, 즉 CNNs입니다. U-Net의 주요 특징은 U자형의 대칭 구조로, 인코딩 경로와 디코딩 경로로 구성되어 있습니다. 인코딩 경로는 입력 이미지를 점점 더 작은 차원으로 축소하고, 디코딩 경로는 이를 다시 원래 크기로 확장하면서 정확한 예측을 만들어냅니다. 인코딩과 디코딩 경로 사이에는 skip connection이 있어, 인코딩 단계에서 추출된 특징을 디코딩 단계에서 다시 사용하여 더 좋은 결과를 얻을 수 있습니다.

### U-Net Architecture

- 인코딩 경로 (Contracting Path):
    - 컨볼루션 레이어와 풀링 레이어를 사용하여 점진적으로 이미지의 차원을 줄입니다.
- 디코딩 경로 (Expansive Path):
    - 업샘플링과 컨볼루션 레이어를 사용하여 이미지의 차원을 원래 크기로 복원합니다.
- Skip Connections:
    - 인코딩 경로의 각 단계에서 얻은 특징 맵을 디코딩 경로의 대응하는 단계에 결합합니다.

### PyTorch Implementation

```python
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms.functional import center_crop

from torchinfo import summary

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        
        # 인코딩 경로 (Contracting Path)
        self.encoder1 = self.contracting_block(in_channels, 64)   # 첫 번째 인코딩 블록: in_channels -> 64 채널
        self.encoder2 = self.contracting_block(64, 128)           # 두 번째 인코딩 블록: 64 -> 128 채널
        self.encoder3 = self.contracting_block(128, 256)          # 세 번째 인코딩 블록: 128 -> 256 채널
        self.encoder4 = self.contracting_block(256, 512)          # 네 번째 인코딩 블록: 256 -> 512 채널
        
        # 중심부 (Bottom of the U)
        self.bottleneck = self.contracting_block(512, 1024)       # 중심부 블록: 512 -> 1024 채널
        
        # 업샘플링 레이어
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)  # 업샘플링: 1024 -> 512 채널
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)   # 업샘플링: 512 -> 256 채널
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)   # 업샘플링: 256 -> 128 채널
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)    # 업샘플링: 128 -> 64 채널
        
        # 디코딩 경로 (Expansive Path)
        self.decoder4 = self.expansive_block(1024, 512)           # 첫 번째 디코딩 블록: 1024 -> 512 채널
        self.decoder3 = self.expansive_block(512, 256)            # 두 번째 디코딩 블록: 512 -> 256 채널
        self.decoder2 = self.expansive_block(256, 128)            # 세 번째 디코딩 블록: 256 -> 128 채널
        self.decoder1 = self.expansive_block(128, 64)             # 네 번째 디코딩 블록: 128 -> 64 채널
        
        # 마지막 출력 레이어
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)  # 최종 출력 레이어: 64 -> out_channels
        
    def contracting_block(self, in_channels, out_channels):
        """
        인코딩 블록 정의: Conv2d -> ReLU -> Conv2d -> ReLU
        """
        block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3),
            nn.ReLU(inplace=True)
        )
        
        return block
    
    def expansive_block(self, in_channels, out_channels):
        """
        디코딩 블록 정의: Conv2d -> ReLU -> Conv2d -> ReLU
        """
        block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3),
            nn.ReLU(inplace=True)
        )
        
        return block
    
    def crop_and_concat(self, upsampled, bypass):
        """
        크기를 맞추기 위해 센터 크롭을 적용하고, 인코딩 경로의 특징 맵과 업샘플링된 특징 맵을 결합
        """
        _, _, H, W = upsampled.size()
        bypass = center_crop(bypass, [H, W])  # 인코딩 경로의 특징 맵을 업샘플링된 맵의 크기에 맞게 Crop
        
        return torch.cat((bypass, upsampled), dim=1)  # 채널 방향으로 결합 (dim=1)
    
    def forward(self, x):
        # 인코딩 경로 (Contracting Path)
        enc1 = self.encoder1(x)                                          # 첫 번째 인코딩 블록
        enc2 = self.encoder2(F.max_pool2d(enc1, kernel_size=2))          # 두 번째 인코딩 블록
        enc3 = self.encoder3(F.max_pool2d(enc2, kernel_size=2))          # 세 번째 인코딩 블록
        enc4 = self.encoder4(F.max_pool2d(enc3, kernel_size=2))          # 네 번째 인코딩 블록
        
        # 중심부 (Bottom of the U)
        bottleneck = self.bottleneck(F.max_pool2d(enc4, kernel_size=2))  # 중심부 블록
        
        # 디코딩 경로 (Expansive Path)
        up4 = self.upconv4(bottleneck)                                   # 업샘플링: 1024 -> 512 채널
        merged4 = self.crop_and_concat(up4, enc4)                        # 인코딩 경로의 특징 맵과 결합
        dec4 = self.decoder4(merged4)                                    # 디코딩 블록 적용
        
        up3 = self.upconv3(dec4)                                         # 업샘플링: 512 -> 256 채널
        merged3 = self.crop_and_concat(up3, enc3)                        # 인코딩 경로의 특징 맵과 결합
        dec3 = self.decoder3(merged3)                                    # 디코딩 블록 적용
        
        up2 = self.upconv2(dec3)                                         # 업샘플링: 256 -> 128 채널
        merged2 = self.crop_and_concat(up2, enc2)                        # 인코딩 경로의 특징 맵과 결합
        dec2 = self.decoder2(merged2)                                    # 디코딩 블록 적용
        
        up1 = self.upconv1(dec2)                                         # 업샘플링: 128 -> 64 채널
        merged1 = self.crop_and_concat(up1, enc1)                        # 인코딩 경로의 특징 맵과 결합
        dec1 = self.decoder1(merged1)                                    # 디코딩 블록 적용
        
        # 최종 출력 레이어
        return self.final_conv(dec1)                                     # 최종 출력 레이어를 통해 결과 반환

# 모델 사용 예시
if __name__ == "__main__":
    model = UNet(in_channels=1, out_channels=2)
    input_tensor = torch.randn(16, 1, 572, 572)  # 입력 크기: (16, 1, 572, 572)
    
    # 모델 구조 요약
    summary(model, input_size=input_tensor.shape, col_width=20, depth=5, row_settings=["depth", "var_names"], col_names=["input_size", "kernel_size", "output_size", "params_percent"])
```